# Building Semantic Memory with Embeddings

So far, we've mostly been treating the kernel as a stateless orchestration engine.
We send text into a model API and receive text out.

In a [previous notebook](04-kernel-arguments-chat.ipynb), we used `kernel arguments` to pass in additional
text into prompts to enrich them with more data. This allowed us to create a basic chat experience.

However, if you solely relied on kernel arguments, you would quickly realize that eventually your prompt
would grow so large that you would run into the model's token limit. What we need is a way to persist state
and build both short-term and long-term memory to empower even more intelligent applications.

To do this, we dive into the key concept of `Semantic Memory` in the Semantic Kernel.


In [ ]:
!python -m pip install semantic-kernel==1.0.5
!python -m pip install azure-core==1.30.1
!python -m pip install azure-search-documents==11.4.0

In [1]:
import sys
import os

# Add the samples directory to the Python path
samples_dir = r'C:\Users\sulma\OneDrive\Documents\GitHub\semantic-kernel\python\samples'
sys.path.append(samples_dir)

try:
    from services import Service
    from service_settings import ServiceSettings

    service_settings = ServiceSettings(
        global_llm_service="AzureOpenAI")  # Example setting

    # Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
    selectedService = (
        Service.AzureOpenAI
        if service_settings.global_llm_service is None
        else Service(service_settings.global_llm_service.lower())
    )
    print(f"Using service type: {selectedService}")
except ModuleNotFoundError as e:
    print(f"Error: {e}")
    print("Ensure that 'service_settings.py' is in the correct directory and properly named.")
    # Additional debugging information
    print("Current Python path:")
    for path in sys.path:
        print(path)

Using service type: Service.AzureOpenAI


In order to use memory, we need to instantiate the Kernel with a Memory Storage
and an Embedding service. In this example, we make use of the `VolatileMemoryStore` which can be thought of as a temporary in-memory storage. This memory is not written to disk and is only available during the app session.

When developing your app you will have the option to plug in persistent storage like Azure AI Search, Azure Cosmos Db, PostgreSQL, SQLite, etc. Semantic Memory allows also to index external data sources, without duplicating all the information as you will see further down in this notebook.


In [2]:
from semantic_kernel import Kernel
from dotenv import load_dotenv
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.connectors.ai.open_ai.services.azure_text_embedding import AzureTextEmbedding
from semantic_kernel.connectors.ai.open_ai.services.open_ai_chat_completion import OpenAIChatCompletion






from semantic_kernel.connectors.ai.open_ai.services.open_ai_text_embedding import OpenAITextEmbedding






from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin






from semantic_kernel.kernel import Kernel






from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory






from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore







kernel = Kernel()







chat_service_id = "chat"


env_path = r'C:\Users\sulma\OneDrive\Documents\GitHub\semantic-kernel\python\samples\getting_started\.env'
load_dotenv(env_path)

service_id = None

# Load the API keys and other settings from the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_org_id = os.getenv('OPENAI_ORG_ID')
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_chat_deployment_name = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
azure_openai_embedding_deployment_name = os.getenv(
    "AZURE_OPENAI_EMBEDDING")



azure_aisearch_api_key = os.getenv('AZURE_AISEARCH_API_KEY')
azure_aisearch_url = os.getenv('AZURE_AISEARCH_URL')






# Configure AI service used by the kernel






if selectedService == Service.AzureOpenAI:






    azure_chat_service = AzureChatCompletion(service_id=chat_service_id,
            api_key=azure_openai_api_key,           # Use the api_key from the .env file
            # Use the deployment_name from the .env file
            deployment_name=azure_chat_deployment_name,
            endpoint=azure_openai_endpoint,         # Use the endpoint from the .env file)






    # next line assumes embeddings deployment name is "text-embedding", adjust the deployment name to the value of your chat model if needed
    )






    embedding_gen = AzureTextEmbedding(
        service_id="embedding", deployment_name=azure_openai_embedding_deployment_name)






    kernel.add_service(azure_chat_service)






    kernel.add_service(embedding_gen)






elif selectedService == Service.OpenAI:






    oai_chat_service = OpenAIChatCompletion(service_id=chat_service_id, ai_model_id="gpt-3.5-turbo")






    embedding_gen = OpenAITextEmbedding(ai_model_id="text-embedding-ada-002")






    kernel.add_service(oai_chat_service)






    kernel.add_service(embedding_gen)







memory = SemanticTextMemory(storage=VolatileMemoryStore(), embeddings_generator=embedding_gen)






kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")

KernelPlugin(name='TextMemoryPlugin', description=None, functions={'recall': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='recall', plugin_name='TextMemoryPlugin', description='Recall a fact from the long term memory', parameters=[KernelParameterMetadata(name='ask', description='The information to retrieve', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'The information to retrieve'}), KernelParameterMetadata(name='collection', description='The collection to search for information.', default_value='generic', type_='str', is_required=False, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'The collection to search for information.'}), KernelParameterMetadata(name='relevance', description='The relevance score, from 0.0 to 1.0; 1.0 means perfect match', default_value=0.75, type_='float', is_required=False, type_object=<class 'float'>, schema_data={'type': 'number', 'descr

At its core, Semantic Memory is a set of data structures that allow you to store the meaning of text that come from different data sources, and optionally to store the source text too. These texts can be from the web, e-mail providers, chats, a database, or from your local directory, and are hooked up to the Semantic Kernel through data source connectors.

The texts are embedded or compressed into a vector of floats representing mathematically the texts' contents and meaning. You can read more about embeddings [here](https://aka.ms/sk/embeddings).


### Manually adding memories

Let's create some initial memories "About Me". We can add memories to our `VolatileMemoryStore` by using `SaveInformationAsync`


In [3]:
collection_id = "generic"


async def populate_memory(memory: SemanticTextMemory) -> None:
    # Add some documents to the semantic memory
    await memory.save_information(collection=collection_id, id="info1", text="Your budget for 2024 is $100,000")
    await memory.save_information(collection=collection_id, id="info2", text="Your savings from 2023 are $50,000")
    await memory.save_information(collection=collection_id, id="info3", text="Your investments are $80,000")

In [4]:
await populate_memory(memory)

Let's try searching the memory:


In [5]:
async def search_memory_examples(memory: SemanticTextMemory) -> None:
    questions = [
        "What is my budget for 2024?",
        "What are my savings from 2023?",
        "What are my investments?",
    ]

    for question in questions:
        print(f"Question: {question}")
        result = await memory.search(collection_id, question)
        print(f"Answer: {result[0].text}\n")

In [6]:
await search_memory_examples(memory)

Question: What is my budget for 2024?
Answer: Your budget for 2024 is $100,000

Question: What are my savings from 2023?
Answer: Your savings from 2023 are $50,000

Question: What are my investments?
Answer: Your investments are $80,000



Let's now revisit the our chat sample from the [previous notebook](04-kernel-arguments-chat.ipynb).
If you remember, we used kernel arguments to fill the prompt with a `history` that continuously got populated as we chatted with the bot. Let's add also memory to it!


This is done by using the `TextMemoryPlugin` which exposes the `recall` native function.

`recall` takes an input ask and performs a similarity search on the contents that have
been embedded in the Memory Store and returns the most relevant memory.


In [7]:
from semantic_kernel.functions import KernelFunction
from semantic_kernel.prompt_template import PromptTemplateConfig


async def setup_chat_with_memory(
    kernel: Kernel,
    service_id: str,
) -> KernelFunction:
    prompt = """
    ChatBot can have a conversation with you about any topic.
    It can give explicit instructions or say 'I don't know' if
    it does not have an answer.

    Information about me, from previous conversations:
    - {{recall 'budget by year'}} What is my budget for 2024?
    - {{recall 'savings from previous year'}} What are my savings from 2023?
    - {{recall 'investments'}} What are my investments?

    {{$request}}
    """.strip()

    prompt_template_config = PromptTemplateConfig(
        template=prompt,
        execution_settings={
            service_id: kernel.get_service(service_id).get_prompt_execution_settings_class()(service_id=service_id)
        },
    )

    return kernel.add_function(
        function_name="chat_with_memory",
        plugin_name="chat",
        prompt_template_config=prompt_template_config,
    )

The `RelevanceParam` is used in memory search and is a measure of the relevance score from 0.0 to 1.0, where 1.0 means a perfect match. We encourage users to experiment with different values.


Now that we've included our memories, let's chat!


In [8]:
print("Populating memory...")
await populate_memory(memory)

print("Asking questions... (manually)")
await search_memory_examples(memory)

print("Setting up a chat (with memory!)")
chat_func = await setup_chat_with_memory(kernel, chat_service_id)

print("Begin chatting (type 'exit' to exit):\n")
print(
    "Welcome to the chat bot!\
    \n  Type 'exit' to exit.\
    \n  Try asking a question about your finances (i.e. \"talk to me about my finances\")."
)


async def chat(user_input: str):
    print(f"User: {user_input}")
    answer = await kernel.invoke(chat_func, request=user_input)
    print(f"ChatBot:> {answer}")

Populating memory...
Asking questions... (manually)
Question: What is my budget for 2024?
Answer: Your budget for 2024 is $100,000

Question: What are my savings from 2023?
Answer: Your savings from 2023 are $50,000

Question: What are my investments?
Answer: Your investments are $80,000

Setting up a chat (with memory!)
Begin chatting (type 'exit' to exit):

Welcome to the chat bot!    
  Type 'exit' to exit.    
  Try asking a question about your finances (i.e. "talk to me about my finances").


In [9]:
await chat("What is my budget for 2024?")

User: What is my budget for 2024?


ChatBot:> Your budget for 2024 is $100,000 as per our previous conversation.


In [10]:
await chat("talk to me about my finances")

User: talk to me about my finances
ChatBot:> Sure! Can you give me more details about your finances? Do you have any specific questions or concerns?


### Adding documents to your memory

Many times in your applications you'll want to bring in external documents into your memory. Let's see how we can do this using our VolatileMemoryStore.

Let's first get some data using some of the links in the Semantic Kernel repo.


In [11]:
github_files = {}
github_files["https://github.com/microsoft/semantic-kernel/blob/main/README.md"] = (
    "README: Installation, getting started, and how to contribute"
)
github_files[
    "https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/02-running-prompts-from-file.ipynb"
] = "Jupyter notebook describing how to pass prompts from a file to a semantic plugin or function"
github_files["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/00-getting-started.ipynb"] = (
    "Jupyter notebook describing how to get started with the Semantic Kernel"
)
github_files["https://github.com/microsoft/semantic-kernel/tree/main/samples/plugins/ChatPlugin/ChatGPT"] = (
    "Sample demonstrating how to create a chat plugin interfacing with ChatGPT"
)
github_files[
    "https://github.com/microsoft/semantic-kernel/blob/main/dotnet/src/SemanticKernel/Memory/Volatile/VolatileMemoryStore.cs"
] = "C# class that defines a volatile embedding store"

Now let's add these files to our VolatileMemoryStore using `SaveReferenceAsync`. We'll separate these memories from the chat memories by putting them in a different collection.


In [12]:
memory_collection_name = "SKGitHub"
print("Adding some GitHub file URLs and their descriptions to a volatile Semantic Memory.")

for index, (entry, value) in enumerate(github_files.items()):
    await memory.save_reference(
        collection=memory_collection_name,
        description=value,
        text=value,
        external_id=entry,
        external_source_name="GitHub",
    )
    print("  URL {} saved".format(index))

Adding some GitHub file URLs and their descriptions to a volatile Semantic Memory.
  URL 0 saved
  URL 1 saved
  URL 2 saved
  URL 3 saved
  URL 4 saved


In [13]:
ask = "I love Jupyter notebooks, how should I get started?"
print("===========================\n" + "Query: " + ask + "\n")

memories = await memory.search(memory_collection_name, ask, limit=5, min_relevance_score=0.77)

for index, memory in enumerate(memories):
    print(f"Result {index}:")
    print("  URL:     : " + memory.id)
    print("  Title    : " + memory.description)
    print("  Relevance: " + str(memory.relevance))
    print()

Query: I love Jupyter notebooks, how should I get started?

Result 0:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/00-getting-started.ipynb
  Title    : Jupyter notebook describing how to get started with the Semantic Kernel
  Relevance: 0.8676695686293272

Result 1:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/README.md
  Title    : README: Installation, getting started, and how to contribute
  Relevance: 0.8080048607377298

Result 2:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/02-running-prompts-from-file.ipynb
  Title    : Jupyter notebook describing how to pass prompts from a file to a semantic plugin or function
  Relevance: 0.8019190676371465



Now you might be wondering what happens if you have so much data that it doesn't fit into your RAM? That's where you want to make use of an external Vector Database made specifically for storing and retrieving embeddings. Fortunately, semantic kernel makes this easy thanks to an extensive list of available connectors. In the following section, we will connect to an existing Azure AI Search service that we will use as an external Vector Database to store and retrieve embeddings.


In [14]:
from semantic_kernel.connectors.memory.azure_cognitive_search import AzureCognitiveSearchMemoryStore

acs_memory_store = AzureCognitiveSearchMemoryStore(
    vector_size=1536, admin_key=azure_aisearch_api_key, search_endpoint = azure_aisearch_url)

memory = SemanticTextMemory(storage=acs_memory_store, embeddings_generator=embedding_gen)
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPluginACS")

KernelPlugin(name='TextMemoryPluginACS', description=None, functions={'recall': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='recall', plugin_name='TextMemoryPluginACS', description='Recall a fact from the long term memory', parameters=[KernelParameterMetadata(name='ask', description='The information to retrieve', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'The information to retrieve'}), KernelParameterMetadata(name='collection', description='The collection to search for information.', default_value='generic', type_='str', is_required=False, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'The collection to search for information.'}), KernelParameterMetadata(name='relevance', description='The relevance score, from 0.0 to 1.0; 1.0 means perfect match', default_value=0.75, type_='float', is_required=False, type_object=<class 'float'>, schema_data={'type': 'number', 

The implementation of Semantic Kernel allows to easily swap memory store for another. Here, we will re-use the functions we initially created for `VolatileMemoryStore` with our new external Vector Store leveraging Azure AI Search


In [15]:
await populate_memory(memory)

Let's now try to query from Azure AI Search!


In [16]:
await search_memory_examples(memory)

Question: What is my budget for 2024?
Answer: Your budget for 2024 is $100,000

Question: What are my savings from 2023?
Answer: Your savings from 2023 are $50,000

Question: What are my investments?
Answer: Your investments are $80,000



We have laid the foundation which will allow us to store an arbitrary amount of data in an external Vector Store above and beyond what could fit in memory at the expense of a little more latency.
